In [69]:
import pandas as pd
import re
import sys
import os

# Añadir la ruta del directorio data_utils al sys.path
# Agregar la carpeta raíz 'data_utils' al sys.path
sys.path.append(os.path.abspath('../'))


In [70]:
#Configuraciones
from data_utils.configuraciones.variables import variables_data

#Manipulacion de datos
from data_utils.manipulacion_de_datos.maniuplacion_de_datos import  procesar_descripcion,   \
                                                                    estandarizado_columnas, \
                                                                    cargar_datos,           \
                                                                    generar_descripcion,    \
                                                                    procesar_descripcion,   \
                                                                    limpiar_datos
from data_utils.manipulacion_de_datos.gestion_de_datos import indexar_productos

#Funciones de Busquedas
from data_utils.funciones_de_busqueda.busqueda_por_filtro import filtrar_por_laboratorio
from data_utils.funciones_de_busqueda.busquedas_por_texto import main_principal

#Aplicaciones de Inteligencia Artificial
from data_utils.utilidades_comunes.aplicaciones_de_inteligencia_artificial.procesamiento_de_lenguaje_natural.index import procesar_comparacion

Instanciamientos de Clases

In [71]:
variables_data = variables_data()


def procesar_descripcion_x(data, nombre_columna):
    """
    Procesa una columna de un DataFrame aplicando normalización y manejo de combinaciones de texto.
    
    Parámetros:
    - data: DataFrame que contiene la columna a procesar.
    - nombre_columna: Nombre de la columna que se va a procesar.
    
    Retorna:
    - DataFrame con la columna procesada y actualizada.
    """
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in data.columns:
        raise KeyError(f"La columna '{nombre_columna}' no existe en el DataFrame.")
    
    # Función para normalizar el texto
    def normalizar(texto):
        # Asegurar que el texto sea un string
        texto = str(texto).strip().lower()
        
        # 1. Eliminar espacios redundantes
        texto = re.sub(r'\s+', ' ', texto)

        # 2. Remover caracteres especiales excepto letras, números, espacios y '%'
        texto = re.sub(r'[^a-zA-Z0-9\s%]', '', texto)

        # 3. Separar números de letras y letras de números
        texto = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', texto)
        texto = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', texto)

        # 4. Manejo de 'x':
        texto = re.sub(r'(?<=\d)x(?=\d)', r' x ', texto)  # separar x entre números
        texto = re.sub(r'\b([a-zA-Z]+)x\b', r'\1 x', texto)  # separar palabras terminadas en x

        # 5. Reemplazar separadores como guiones o barras con espacios
        texto = re.sub(r'[-/]', ' ', texto)

        # 6. Manejo de unidades (por ejemplo, separar g, ml, etc. de los números)
        texto = re.sub(r'(\d+)\s*([a-zA-Z]+)', r'\1 \2', texto)

        # 7. Eliminar espacios redundantes nuevamente
        texto = texto.strip()

        # 8. Combinaciones
        combinaciones = {
            'anemido x': 'anemidox',
            'jgaprell' : 'jeringa prellenada',
            'fle x': 'flex',
            'argeflo x':'argeflox',
            'aropa x': 'aropax',
            'arolte x' : 'aroltex',
            'atopi x':  'atopix',
            'atorma x': 'atormax',
            'comprec': 'comp recubierto',
            'soloft': 'solucion oftalmica',
            'g': 'gr',
            'cr': 'crema',
            'comp' : 'comprimidos',
            'caps': 'capsulas',
            "jer.pre" : "jeringa prellenada",
            "mg" : "miligramos",
            "ml" : "mililitros",
            "mm" : "milimetros",
            'zyvo x': 'zyvox',
            'loc' : 'locion',
            "gr" : "gramos",
            "jga" : 'jeringa',
            "cm" : 'centimetros',
            'u' : "unidad",
            'shamp' : 'shampoo',
            "unid" : " unidad",
            'cep' : 'cepillo dental',
            "emuls prot" : 'emulsion protectora',
            'emulshidrat':'emulsion hidratante',
            'emulshumect' : "emulsion humectante",
            'iny': 'inyeccion',
            'amp' : 'ampolla',
            'inylioffa': 'inyeccion lioffa',
            'ds' : 'dosis',
            'inyfa': 'inyeccion fa',
            'jbe': 'jarabe',
            'blist': 'blister',
            'sob' : 'sobres',
            'fa': 'frasco ampolla',
            'jab' : 'jabon',
            'cps' :'capsulas',
            'ivi x' : 'ivix',
            'inya': 'inyectable',
            'inyjgaprell' : 'iny jeringa prellenada',
            'mcg': 'microgramo',
            'gts': 'gotas',
            'mgml' : 'miligramos por mililitro',
            'env': 'envase',
            'gtsoft' : 'gotas oftálmicas',
            'compdisp': 'comprimidos disp',
            'comprecran' : 'comprimidos recubiertos ranurados',
            'sol': 'solucion',
            'jgapre': 'jeringa prellenada',
            'fcocps' : 'frasco capsulas',
            'fcogotero' : 'frasco gotero',
            'fco': 'frasco', 
            'solspray' : 'solucion spray',
            'solpuas' : 'sol puas',
            'cpsblandas' : 'cps blandas',
            'tab' : 'tableta',
            'sach' : 'sachet',
            'mgvial': 'miligramos vial',
            'cpsbl' :'capsula blanda',
            'zyvali x': 'zyvalix',
            'x' : '',
            'lapprell': 'lapicera prellenada',
            'complibprol' : 'comp lib prol ',
            'compcu': 'comp cu',
            'jerprell'  : "jeringa prellenada",
            'prell': 'prellenada'    
        }
        for key, val in combinaciones.items():
            texto = re.sub(rf'\b{key}\b', val, texto)

        return texto

    # Aplicar la función 'normalizar' a cada elemento de la columna
    data[nombre_columna] = data[nombre_columna].apply(normalizar)
    
    return data 

Procesamiento df Master Producto

In [72]:
#Carga de Producto
df_Producto = cargar_datos(variables_data.csv_producto)
df_Producto = limpiar_datos(df_Producto)
df_Producto = indexar_productos(df_Producto,variables_data.index_productos)
df_MasterProductos = generar_descripcion(df_Producto,variables_data.name_columna_1_MasterProductos,variables_data.name_columna_2_MasterProductos,variables_data.descripcion_limpia_MasterProductos)
df_MasterProductos = procesar_descripcion_x(df_MasterProductos, variables_data.descripcion_limpia_MasterProductos)
df_MasterProductos['descripcion_limpia_producto']

c:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\data_utils\manipulacion_de_datos\maniuplacion_de_datos.py:15: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(ruta_csv)


0                                                         
1                                                         
2        repetido supportan 500 mililitros caja  15 sac...
3        bolsas papel grado medico 175  65  420 milimet...
4        bolsas papel grado medico 260  60  520 milimet...
                               ...                        
56769              zyvalix 250 miligramos comprimidos  120
56770          zyvalix plus 500 miligramos comprimidos  60
56771    zyvox iv 600 miligramos bolsas  10  300 milili...
56772                zyvox oral 600 miligramos tableta  10
56773                               zz top comprimidos  30
Name: descripcion_limpia_producto, Length: 56770, dtype: object

DataFrame Master Producto Filtrado por Laboratorio

In [73]:
# El usuario ingresa el nombre del laboratorio
laboratorio_input = input("Ingresa el nombre del laboratorio: ").strip()
# Filtrar productos por laboratorio
df_MasterProductos_Filtro = filtrar_por_laboratorio(df_MasterProductos, laboratorio_input)

Se encontraron 38 productos coincidentes del laboratorio 'ABBVIE'.


DataFrame Proveedor

In [74]:
#Carga Proveedor
cvs_proveedor = 'Fabra.csv'
df_Proveedor = cargar_datos(cvs_proveedor)
# Eliminar filas donde todas sus columnas están vacías
df_Proveedor = limpiar_datos(df_Proveedor)
#Indexacion 
df_Proveedor = indexar_productos(df_Proveedor,variables_data.index_proveedor)
#Normalizacion de Columnas
df_Proveedor = estandarizado_columnas(df_Proveedor,variables_data.name_columna_1_Proveedor,variables_data.name_columna_2_Proveedor,variables_data.descripcion_limpia_Proveedor)
#Normalizacion de Columna Descripcion
df_Proveedor = procesar_descripcion_x(df_Proveedor, variables_data.descripcion_limpia_Proveedor)
df_Proveedor.columns

Index(['principio activo', 'n°cert.', 'descripcion', 'presentacion', 'bli',
       'cert.', 'caja', 'caja', 'pc sin iva', 'pc lista', 'desc', ' pc desc ',
       'index_proveedor', 'descripcion_limpia_proveedor'],
      dtype='object')

Procesamiento CODIGO DE PRODUCTO

In [75]:
df_MasterProductos_Filtro.columns

Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto'],
      dtype='object')

In [76]:
df_Proveedor.head(10)

,principio activo,n°cert.,descripcion,presentacion,bli,cert.,caja,caja,pc sin iva,pc lista,desc,pc desc,index_proveedor,descripcion_limpia_proveedor
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lista,NaN,NaN,NaN,0,
1,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,#REF!,NaN,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros
2,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,"$ 371,901","$ 450,00",20%,"$ 360,00",2,agua destilada 5 mililitros ampolla
3,ALLOPURINOL 300 mg,41.305,ALLOPURINOL FABRA,Comp.,10,41305.0,500,4000,"$ 211,777","$ 256,25",20%,"$ 205,00",3,allopurinol fabra comprimidos
4,AMIKACINA 100 mg,43.826,AMIKACINA FABRA,Amp 2 ml,NaN,43826.0,100,1200,"$ 645,661","$ 781,25",20%,"$ 625,00",4,amikacina fabra ampolla 2 mililitros
5,AMIKACINA 500 mg,43.826,AMIKACINA FABRA,Amp 2 ml,NaN,43826.0,100,1200,"$ 1.301,653","$ 1.575,00",20%,"$ 1.260,00",5,amikacina fabra ampolla 2 mililitros
6,AMINOFILINA 240 mg/ml x 10 ml,38.930,AMINOFILINA FABRA,Amp,NaN,NaN,100,1200,$ -,$ -,20%,$ -,6,aminofilina fabra ampolla
7,AMIODARONA 200 mg,41.867,AMIODARONA FABRA,Comp.,10,41867.0,500,6000,"$ 268,595","$ 325,00",20%,"$ 260,00",7,amiodarona fabra comprimidos
8,AMLODIPINA 5 mg.,44.076,ANGIOFILINA 5 mg.,Comp.,10,44076.0,500,6000,"$ 99,174","$ 120,00",20%,"$ 96,00",8,angiofilina 5 miligramos comprimidos
9,AMLODIPINA 10 mg,44.076,ANGIOFILINA 10 mg.,Comp.,10,44076.0,500,6000,"$ 109,504","$ 132,50",20%,"$ 106,00",9,angiofilina 10 miligramos comprimidos


Procesamiento NLP DESCRIPCION

In [77]:
df_resultado_similiridad = main_principal(df_MasterProductos_Filtro, df_Proveedor)
#df_resultado_similiridad

Procesamiento para comparacion de  Descripciones

In [78]:
df_resultado_similiridad[df_resultado_similiridad['similaridad'] > 0.8][['similaridad', 'niprod', 'cod_prod', 'descripcion_limpia_proveedor', 'descripcion_limpia_producto', 'index_proveedor_y','codbarra']]

,similaridad,niprod,cod_prod,descripcion_limpia_proveedor,descripcion_limpia_producto,index_proveedor_y,codbarra


INDEX DE SUSTRACCION PARA EL DF PROVEEDOR

In [87]:
#Sutraccion de productos sobre df ORIGINAL DE PROVEEDORES.
similiridad = 'similaridad'

umbral_similaridad = 0.9  # Ajusta este valor según tu caso
df_filtrado_similaridad = df_resultado_similiridad[df_resultado_similiridad[similiridad] > umbral_similaridad]

# Ahora, con los índices originales, extraemos los productos del df_Proveedor
# Filtramos por 'indice_original_proveedor' para obtener solo las filas que superaron el umbral
df_Proveedor_Similiridad_filtrado = df_Proveedor[df_Proveedor[variables_data.index_proveedor].isin(df_filtrado_similaridad['index_proveedor_x'])]

# Mostrar los productos filtrados del proveedor
df_Proveedor_Similiridad_filtrado


,principio activo,n°cert.,descripcion,presentacion,bli,cert.,caja,caja,pc sin iva,pc lista,desc,pc desc,index_proveedor,descripcion_limpia_proveedor,numeros


In [88]:
df_MasterProductos.columns

#Arquitectura df_Productos
 
# Niprod = Sistema Carena
# ean = ????
# descrip1 = Descripcion Provedor
# descrip2 = Presentacion Proveedor
# atrib0   = laboratorio
# codbarra = codbarra
#cod_prod  = codigo_producto_laboratorio

Index(['niprod', 'cod_prod', 'cod_abrev', 'descrip1', 'descrip2', 'abrev',
       'cod_catprod', 'codbarra', 'cod_fbarra', 'unid_v', 'base_v', 'precio',
       'mon_v', 'costo', 'base_c', 'unid_c', 'mon_c', 'rel_c_v', 'rel_v_c',
       'unid_alt', 'rel_c_a', 'rel_a_c', 'cod_mone', 'habilitado', 'cod_bonif',
       'pped', 'st_min', 'st_max', 'observ', 'pc_comision', 'pc_utilidad',
       'ind_stock', 'ind_partidas', 'ind_camestado', 'ind_series', 'ind_fvto',
       'ninum', 'tipo_egreso', 'pc_diftrans', 'cod_cvta', 'cat_item',
       'cant_rep', 'pto_rep', 'atrib0', 'atrib1', 'atrib2', 'index_producto',
       'descripcion_limpia_producto'],
      dtype='object')

MODELO DE PROCESAMIENTOS DE LENGUAJE NATURAL

Bert

In [89]:

modelo_nombre_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

resultados_comparaciones = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BERT,                # modelo_nombre
    "BERT",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\data_utils\utilidades_comunes\aplicaciones_de_inteligencia_artificial\procesamiento_de_lenguaje_natural\index.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_master['numeros'] = df_master[columna_master].apply(extraer_numeros)  # Extrae números del master
c:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\data_utils

In [90]:
resultados_comparaciones.head(10)


,descripcion_proveedor,descripcion_match,similitud,index_proveedor,index_producto,niprod
0,,viekira pak 28 blis 4 comp cu,0.433874,0,54849,119576.0
1,,maviret comprimidos recubierto 84,0.423680,0,33496,119558.0
2,,zemplar 2 microgramo capsulas 30,0.420144,0,56457,119577.0
3,agua destilada fabra ampolla 5 mililitros,zemplar 5 microgramo vial 5 1 mililitros,0.800929,1,56459,119579.0
4,agua destilada fabra ampolla 5 mililitros,simda 25 miligramos por mililitro inyeccion f...,0.799820,1,47110,119564.0
5,agua destilada fabra ampolla 5 mililitros,zemplar 5 microgramo a 5 1 mililitros,0.792686,1,56458,119578.0
6,agua destilada 5 mililitros ampolla,simda 25 miligramos por mililitro inyeccion f...,0.811538,2,47110,119564.0
7,agua destilada 5 mililitros ampolla,epkinly 5 miligramos por mililitro solucion in...,0.788734,2,19970,134919.0
8,agua destilada 5 mililitros ampolla,zemplar 5 microgramo vial 5 1 mililitros,0.786160,2,56459,119579.0
9,allopurinol fabra comprimidos,maviret comprimidos recubierto 84,0.742072,3,33496,119558.0


In [91]:
# Unir df_Proveedor con resultados_comparaciones en base a index_proveedor
df_resultado = df_Proveedor.merge(resultados_comparaciones, 
                                  left_index=True,  # Índice de df_Proveedor (index_proveedor debe coincidir con el índice)
                                  right_on='index_proveedor',  # Columna en resultados_comparaciones
                                  how='right')  # Solo los que están en resultados_comparaciones

# Reorganizar las columnas si es necesario
df_resultado = df_resultado[df_Proveedor.columns.tolist() + ['index_producto', 'niprod', 'similitud']]

# Mostrar el resultado
df_resultado.head(15)


,principio activo,n°cert.,descripcion,presentacion,bli,cert.,caja,caja,caja,caja,pc sin iva,pc lista,desc,pc desc,index_proveedor,descripcion_limpia_proveedor,numeros,index_producto,niprod,similitud
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lista,NaN,NaN,NaN,0,,[],54849,119576.0,0.433874
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lista,NaN,NaN,NaN,0,,[],33496,119558.0,0.423680
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lista,NaN,NaN,NaN,0,,[],56457,119577.0,0.420144
3,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,100,800,#REF!,NaN,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros,[5],56459,119579.0,0.800929
4,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,100,800,#REF!,NaN,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros,[5],47110,119564.0,0.799820
5,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,100,800,#REF!,NaN,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros,[5],56458,119578.0,0.792686
6,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,100,800,"$ 371,901","$ 450,00",20%,"$ 360,00",2,agua destilada 5 mililitros ampolla,[5],47110,119564.0,0.811538
7,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,100,800,"$ 371,901","$ 450,00",20%,"$ 360,00",2,agua destilada 5 mililitros ampolla,[5],19970,134919.0,0.788734
8,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,100,800,"$ 371,901","$ 450,00",20%,"$ 360,00",2,agua destilada 5 mililitros ampolla,[5],56459,119579.0,0.786160
9,ALLOPURINOL 300 mg,41.305,ALLOPURINOL FABRA,Comp.,10,41305.0,500,4000,500,4000,"$ 211,777","$ 256,25",20%,"$ 205,00",3,allopurinol fabra comprimidos,[],33496,119558.0,0.742072


In [93]:
# Unir df_resultado con masterProductos en base a la columna niprod
df_resultado = df_resultado.merge(df_MasterProductos_Filtro[['niprod', variables_data.descripcion_limpia_MasterProductos]], 
                                  on='niprod',  # Clave de unión
                                  how='left')  # 'left' para no perder filas en df_resultado

# Mostrar el resultado con la nueva columna agregada
df_resultado.head(15)


,principio activo,n°cert.,descripcion,presentacion,bli,cert.,caja,caja,caja,caja,...,desc,pc desc,index_proveedor,descripcion_limpia_proveedor,numeros,index_producto,niprod,similitud,descripcion_limpia_producto_x,descripcion_limpia_producto_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,,[],54849,119576.0,0.433874,viekira pak 28 blis 4 comp cu,viekira pak 28 blis 4 comp cu
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,,[],33496,119558.0,0.423680,maviret comprimidos recubierto 84,maviret comprimidos recubierto 84
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,,[],56457,119577.0,0.420144,zemplar 2 microgramo capsulas 30,zemplar 2 microgramo capsulas 30
3,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,100,800,...,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros,[5],56459,119579.0,0.800929,zemplar 5 microgramo vial 5 1 mililitros,zemplar 5 microgramo vial 5 1 mililitros
4,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,100,800,...,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros,[5],47110,119564.0,0.799820,simda 25 miligramos por mililitro inyeccion f...,simda 25 miligramos por mililitro inyeccion f...
5,AGUA DESTILADA,45.767,AGUA DESTILADA FABRA,Amp 5 ml,NaN,NaN,100,800,100,800,...,NaN,$ -,1,agua destilada fabra ampolla 5 mililitros,[5],56458,119578.0,0.792686,zemplar 5 microgramo a 5 1 mililitros,zemplar 5 microgramo a 5 1 mililitros
6,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,100,800,...,20%,"$ 360,00",2,agua destilada 5 mililitros ampolla,[5],47110,119564.0,0.811538,simda 25 miligramos por mililitro inyeccion f...,simda 25 miligramos por mililitro inyeccion f...
7,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,100,800,...,20%,"$ 360,00",2,agua destilada 5 mililitros ampolla,[5],19970,134919.0,0.788734,epkinly 5 miligramos por mililitro solucion in...,epkinly 5 miligramos por mililitro solucion in...
8,AGUA DESTILADA x 5 ml,45.767,AGUA DESTILADA x 5 ml,Amp,NaN,45767.0,100,800,100,800,...,20%,"$ 360,00",2,agua destilada 5 mililitros ampolla,[5],56459,119579.0,0.786160,zemplar 5 microgramo vial 5 1 mililitros,zemplar 5 microgramo vial 5 1 mililitros
9,ALLOPURINOL 300 mg,41.305,ALLOPURINOL FABRA,Comp.,10,41305.0,500,4000,500,4000,...,20%,"$ 205,00",3,allopurinol fabra comprimidos,[],33496,119558.0,0.742072,maviret comprimidos recubierto 84,maviret comprimidos recubierto 84


PubMedBERT

In [85]:
"""
modelo_nombre_PubMedBert = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract'

resultados_PubMedBERT = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_PubMedBert,                # modelo_nombre
    "PubMedBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
"""

'\nmodelo_nombre_PubMedBert = \'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract\'\n\nresultados_PubMedBERT = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_PubMedBert,                # modelo_nombre\n    "PubMedBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n'

BioBert

In [86]:
"""
modelo_nombre_BioBert = 'dmis-lab/biobert-base-cased-v1.1'

resultados_BioBert = procesar_comparacion(
    df_MasterProductos_Filtro, 
    df_Proveedor, 
    modelo_nombre_BioBert,                # modelo_nombre
    "BioBert",                # nombre_modelo
    'descripcion_limpia_producto',  # columna_master
    'descripcion_limpia_proveedor'  # columna_proveedor
)
 """

'\nmodelo_nombre_BioBert = \'dmis-lab/biobert-base-cased-v1.1\'\n\nresultados_BioBert = procesar_comparacion(\n    df_MasterProductos_Filtro, \n    df_Proveedor, \n    modelo_nombre_BioBert,                # modelo_nombre\n    "BioBert",                # nombre_modelo\n    \'descripcion_limpia_producto\',  # columna_master\n    \'descripcion_limpia_proveedor\'  # columna_proveedor\n)\n '